# Facenet Tutorial

Hey guys! Today we're going to use facnet_pytorch to help you with some handy-dandy face recognition. Facenet-pytorch is a [PyTorch](https://pytorch.org/) implementation of [this paper](https://arxiv.org/abs/1503.03832) *FaceNet: A unified Embedding for Face Recognition and Clustering*, which achieves state-of-the-art accuracy in face recognition.

**Why are we using facenet-pytorch?** You might be thinking, why not just work on another neural net to learn to detect and recognize faces? Two things: data, and time. In order to achieve a model with accuracy as good as facenet-pytorch, we'd need tons of labeled data that's pretty hard to come by. And, to create and train a model of that magnitude would take much more time than we have avaliable. However, you all already know the basic concepts of what's going on under the hood, and it's more convienent and accurate to use this model.

First, you're going to want to clone and do the whole `python setup.py develop` thing to use the facenet models - you can find the repo [here](https://github.com/CogWorksBWSI/facenet_models).

Make sure you've already run through the Camera Tutorial module to set up your camera and to check if the ports are correct, etc! (you can find Camera [here](https://github.com/LLCogWorks2018/Camera) if you haven't set it up yet) 

In [ ]:
# run this cell to setup matplotlib, and also import the very important take_picture function from camera!
%matplotlib notebook
import matplotlib.pyplot as plt
from camera import take_picture
import numpy as np

Let's take a picture to test this out!

In [ ]:
fig,ax = plt.subplots()
pic = take_picture()
ax.imshow(pic)

Hello gorgeous!! Now, let's see if we can detect any faces.

In [ ]:
from facenet_models import FacenetModel

model = FacenetModel()

In [ ]:
boxes, probabilities, landmarks = model.detect(pic)

print(f"Number of faces detected: {len(boxes)}")

Heck yeah, we got a face, which is a step in the right direction. The facenet-pytorch model gives us a set of boxes containing the face regions of our image and a set of landmarks, which correspond to specific facial attributes: your eyes, corners of your mouth, and nose. Let's visualize those features now!

In [ ]:
from matplotlib.patches import Rectangle
fig,ax = plt.subplots()
ax.imshow(pic)


for box, prob, landmark in zip(boxes, probabilities, landmarks):
    # draw the box on the screen
    ax.add_patch(Rectangle(box[:2], *(box[2:] - box[:2]), fill=None, lw=2, color="red"))

    # Get the landmarks/parts for the face in box d.
    # Draw the face landmarks on the screen.
    for i in range(len(landmark)):
        ax.plot(landmark[i, 0], landmark[i, 1], '+', color="blue")

We're able to find faces in an image! However, in order to match names with faces, we need to be able to distinguish *between* faces. Luckily, our model has this cool feature called a "face descriptor vector". Think of a face descriptor as a numerical representation of the intricacies of your face. We'll tell the model where the faces are by passing in the boxes and it will give us the face descriptors, which are 512-dimensional vectors encoding a person's identity.

In [ ]:
# let's take a look as to what the descriptor is!!
descriptor = model.compute_descriptors(pic, boxes)
print(descriptor.shape)
print(descriptor)

Woah, that's pretty cool!! As you can see, the **descriptor vector** looks like a pretty standard NumPy array. Each descriptor vector is different. Even if we take descriptor vectors of the same face in two different pictures, they'll be slightly different. Now, the problem lies in how we can distinguish between the minute details in faces of the same people and the large differences in other faces. 

There's a pretty tried and true formula for finding this out, and it should look a little familiar. We can calculate the distance between each point in the vector, add it all up, and see if it's greater or less than a certain threshold value that we can determine ourselves.

differences = $\sqrt{\sum{((p1 - p2)^2)}}$

As a note, we square each difference between points to make sure it isn't negative and messing up our total. Then, we square root it all at the end to even it back out.

Alright, now it's your turn! Grab a friend and take a picture. As a note, you'll get two detections, two shapes, and two descriptor values, so remember to account for them both. Try seeing how different your faces are. Then, maybe take two pictures of yourself and see the difference between the two. Try to come up with a good threshold value between determining different people. 

Have fun!